## Kaggle Submission - Checking model against Test data set

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV 

In [19]:
# Load test data from test.csv in datasets folder
kaggle_test = pd.read_csv('./datasets/test.csv')

In [20]:
# Load Lasso X Train transformed data from CSV file
XLs_train = pd.read_csv('./datasets/XLs_test_transformed.csv')

# Load Lasso y train data from CSV file
yl_train = pd.read_csv('./datasets/yl_train.csv')
yl_train = yl_train['SalePrice']

In [35]:
lcv = LassoCV(alphas=[np.logspace(-3, 0, 100)], cv=5, max_iter=50000);
lcv.fit(XLs_train, yl_train);

In [25]:
# Hot code categorical features 
features_to_change = list(kaggle_test.select_dtypes([np.object]).columns)
kaggle_test = pd.get_dummies(kaggle_test, columns=features_to_change, drop_first=True)

In [26]:
# Get the lasso columns from X train
kaggle_features = list(XLs_train.columns)

In [27]:
X_TEST = kaggle_test[kaggle_features]

In [28]:
# Check for Null Values
X_TEST.isnull().sum()[X_TEST.isnull().sum()>0]

Lot Frontage    160
dtype: int64

In [29]:
# Convert the null values in Lot Frontage to 0
X_TEST['Lot Frontage'] = X_TEST['Lot Frontage'].map(lambda cell: 0 if pd.isnull(cell) else cell)

<ipython-input-29-e6714ef47505>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_TEST['Lot Frontage'] = X_TEST['Lot Frontage'].map(lambda cell: 0 if pd.isnull(cell) else cell)


In [30]:
# Scale data
ss = StandardScaler()
Xs_TEST = ss.fit_transform(X_TEST)

In [31]:
kaggle_test['y_hat'] = lcv.predict(Xs_TEST)

In [32]:
submission = pd.DataFrame({'Id': kaggle_test['Id'],
                          'SalePrice': kaggle_test['y_hat']})

In [33]:
# Check size of output file to the template in Kaggle
submission.shape

(878, 2)

In [34]:
# Export to CSV
submission.to_csv('AC_submission11.csv', index=False)